<a href="https://colab.research.google.com/github/rg-smith/remote_sensing_course/blob/main/lectures/lecture10/time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Time Series Analysis
This code was modified from the Google Earth Engine python api tutorial created by guiattard (https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)

First, we will initialize Google Earth Engine

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Now, we will gather the image collections we are using.

In [18]:
# Initial date of interest (inclusive).
i_date = '2017-01-01'

# Final date of interest (exclusive).
f_date = '2020-01-01'

# Import the MODIS land cover collection.
lc = ee.ImageCollection('MODIS/006/MCD12Q1')

# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection('MODIS/006/MOD11A2')
lst = lst.select('LST_Day_1km','QC_Day').filterDate(i_date,f_date)

# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

ndvi = ee.ImageCollection('MODIS/006/MOD13A2')
ndvi = ndvi.filterDate(i_date,f_date)

**Input the coordinates of your area of interest. Replace the coordinates in u_lon and u_lat**

In [19]:
# Define the urban location of interest as a point near Lyon, France.
u_lon = 4.8148
u_lat = 45.7758
u_poi = ee.Geometry.Point(u_lon, u_lat)

Calculate average values over your area of interest. If you are above 60 degrees north (or below 60 degrees south), comment out the elevation lines.

In [ ]:
scale = 1000  # scale in meters

# Print the elevation near Lyon, France.
elv_urban_point = elv.sample(u_poi, scale).first().get('elevation').getInfo()
print('Ground elevation at point:', elv_urban_point, 'm')

# Calculate and print the mean value of the LST collection at the point.
lst_urban_point = lst.mean().sample(u_poi, scale).first().get('LST_Day_1km').getInfo()
print('Average daytime LST at point:', round(lst_urban_point*0.02 -273.15, 2), '°C')

# Print the land cover type at the point.
lc_urban_point = lc.first().sample(u_poi, scale).first().get('LC_Type1').getInfo()
print('Land cover value at point is:', lc_urban_point)

# Calculate and Print the mean NDVI at the point.
ndvi_urban_point = ndvi.mean().sample(u_poi, scale).first().get('NDVI').getInfo()
print('Land cover value at point is:', lc_urban_point)

Now you will extract the data over all times from these points.

In [ ]:
# The buffer zone we consider around each point.
point_buffer = 1000  # meters

# Get the temperature data for the point of interest.
lst_u_poi = lst.getRegion(u_poi, point_buffer).getInfo()

# Get the ndvi data for the point of interest
ndvi_u_poi = ndvi.getRegion(u_poi, point_buffer).getInfo()

# Preview the result.
lst_u_poi[:5]

Define a function that transforms your google earth engine dataset to a data frame.

In [22]:
import pandas as pd

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

Use your function to convert temperature and ndvi to a dataframe.

In [ ]:
lst_df_urban = ee_array_to_df(lst_u_poi,['LST_Day_1km'])
ndvi_df = ee_array_to_df(ndvi_u_poi,['NDVI'])

def t_modis_to_celsius(t_modis):
    """Converts MODIS LST units to degrees Celsius."""
    t_celsius =  0.02*t_modis - 273.15
    return t_celsius

# Apply the function to get temperature in celsius.
lst_df_urban['LST_Day_1km'] = lst_df_urban['LST_Day_1km'].apply(t_modis_to_celsius)

lst_df_urban.head()
ndvi_df.head()

Plot the temperature over time

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## First, extract x values (times) from the dfs.
x_data_u = np.asanyarray(lst_df_urban['time'].apply(float))  # urban

## Secondly, extract y values (LST) from the dfs.
y_data_u = np.asanyarray(lst_df_urban['LST_Day_1km'].apply(float))  # urban

# Subplots.
fig, ax = plt.subplots(figsize=(14, 6))

# Add scatter plots.
ax.scatter(lst_df_urban['datetime'], lst_df_urban['LST_Day_1km'],
           c='black', alpha=0.2, label='Temperature [C]')

# Add some parameters.
ax.set_title('Daytime Land Surface Temperature', fontsize=16)
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Temperature [C]', fontsize=14)
ax.set_ylim(-0, 40)
ax.grid(lw=0.2)
ax.legend(fontsize=14, loc='lower right')

plt.show()

Plot the ndvi over time

In [ ]:
## First, extract x values (times) from the dfs.
x_data_u = np.asanyarray(ndvi_df['time'].apply(float))  

## Secondly, extract y values (LST) from the dfs.
y_data_u = np.asanyarray(ndvi_df['NDVI'].apply(float))  

# Subplots.
fig, ax = plt.subplots(figsize=(14, 6))

# Add scatter plots.
ax.scatter(ndvi_df['datetime'], ndvi_df['NDVI'],
           c='black', alpha=0.2, label='NDVI')

# Add some parameters.
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('NDVI', fontsize=14)
ax.grid(lw=0.2)
ax.legend(fontsize=14, loc='lower right')

plt.show()